# astroHACK VLA Tutorial




## Download Tutorial Data

In [ ]:
from astrohack.gdown_utils import gdown_data
gdown_data('ea25_cal_small_spw1_4_60_ea04_after.ms',download_folder='data')

## Setup Dask Local Cluster

In [ ]:
from astrohack.astrohack_client import astrohack_local_client

client = astrohack_local_client(cores=2,memory_limit='8GB')
client

## Extract Holog

In [ ]:
from astrohack.extract_holog import extract_holog
scans=[8,9,10,12,13,14,16,17,18,23,24,25,27,28,29,31,32,33,38,39,40,42,43,44,46,47,48,53,54,55,57]
holog_obs_description = {'map_0' :{'scans':scans,'ant':{'ea25':['ea04']}}} #change 0 -> map_0
holog_obs_description['ddi'] = [0]

holog_mds = extract_holog(ms_name='data/ea25_cal_small_spw1_4_60_ea04_after.ms', 
                                     holog_obs_dict=holog_obs_description,
                                     data_col='CORRECTED_DATA',
                                     parallel=True,
                                     overwrite=True,
                                    )

The holog_mds object is a python dict that has extended with the following functionality 

In [ ]:
holog_mds.summary()

In [ ]:
from astrohack._utils._dio_classes import AstrohackHologFile

holog_mds = AstrohackHologFile('data/ea25_cal_small_spw1_4_60_ea04_after.holog.zarr')
holog_mds.open()
ds = holog_mds['ddi_0']['map_0']['ant_ea25']

holog_correct_mds = AstrohackHologFile('data/correct/ea25_cal_small_spw1_4_60_ea04_after.holog.zarr')
holog_correct_mds.open()
c_ds = holog_correct_mds['ddi_0']['map_0']['ant_23']

In [ ]:
c_ds

In [ ]:
ds

## Holog

In [ ]:
from astrohack import holog
import numpy as np

cell_size = np.array([-0.0006442, 0.0006442]) #arcseconds
grid_size = np.array([31, 31]) #pixels

image_mds = holog(holog_name='data/ea25_cal_small_spw1_4_60_ea04_after.holog.zarr',
                  grid_size=grid_size, cell_size=cell_size, 
                  overwrite=True,
                  phase_fit=True,
                  apply_mask=True,
                  to_stokes=True,
                  parallel=True
                 )

In [ ]:
image_mds.summary()

In [ ]:
image_mds['ant_ea25']['ddi_0']

In [ ]:
image_mds['ant_ea25']['ddi_0'].ANGLE.isel(chan=0,pol=0).plot()

In [ ]:
from astrohack._utils._dio_classes import AstrohackImageFile

image_mds = AstrohackImageFile('data/ea25_cal_small_spw1_4_60_ea04_after.image.zarr')
image_mds.open()
image_mds.summary()

## Panel

In [ ]:
from astrohack.panel import panel
panel_model = 'rigid'

panel_mds = panel(
                    image_name='data/ea25_cal_small_spw1_4_60_ea04_after.image.zarr', 
                    panel_model=panel_model,
                    unit='mils', 
                    parallel=True,
                    overwrite=True
                )

panel_mds

In [ ]:
from astrohack._utils._dio_classes import AstrohackPanelFile

a = AstrohackPanelFile('data/ea25_cal_small_spw1_4_60_ea04_after.panel.zarr')
a.open()

In [ ]:
a.summary()